In [2]:
import ast
import os
import re
from pprint import pprint
from tqdm import tqdm
from parsing import get_inline_citations
from utils import load_dataset

HF_TOKEN = os.getenv("HF_ACCESS_TOKEN")

astro_reviews = load_dataset("data/sentence_segmented/Astro_Reviews.json")
citations = get_inline_citations(astro_reviews[0]['body'])
print(len(citations))

data/sentence_segmented/Astro_Reviews.json: 996/996 have all required keys
352


In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="meta-llama/Llama-3.3-70B-Instruct", token=HF_TOKEN)
pipe(messages)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
astro_reviews = load_dataset("data/sentence_segmented/Astro_Reviews.json")

In [ ]:
prompt = """
Extract and output ONLY the inline citations from the text below as a list of tuples
- Each citation becomes a (string, int) tuple where the string is the first author's name and the int is the year
- If there are no citations in the text, output []
- Do not count citations 'in preparation' or lacking a year
- Do not include any introductory text, explanations, or anything before or after the array

Examples of inline citations:
'''
Sentence: "Like Caffau et al. (2008a) , we have similar findings."
Output: [('Caffau et al.', 2008)]

Sentence: "Methods for mixing below the convection zone are well understood ( Brun, Turck-Chièze Zahn 1999 , Charbonnel Talon 2005 )."
Output: [('Brun', 1999), ('Charbonnel', 2005)]

Sentence: "Momentum balance gives an expression ( Fabian 1999 ; Fabian, Wilman Crawford 2002 ; King 2003 , 2005 )"
Output: [('Fabian', 1999), ('Fabian', 2002), ('King', 2003), ('King', 2005)]

Sentence: "This is consistent with previous results (Pereira et al., in preparation)."
Output: []
'''

Now extract the inline citations from the following text:
'''
{text}
'''

Output format: 
[('first author', year), ('first author', year), ...]
"""
text = 'neglect the H collisions altogether based on the available atomic physics data for other elements, while others use the classical Drawin (1968) formula, possibly with a scaling factor S H that typically varies from 0 to 1. Holweger (2001) found log ε O = 8.71 ± 0.05 using the Holweger Müller (1974) model with granulation corrections'


def create_prompt_batch(prompt, texts):
    return [prompt.format(text=text) for text in texts]
    

prompts = create_prompt_batch(prompt, [text, text])
results = pipe(prompts)
